Importar librerias

In [ ]:
import os
from src.config import *
from src.data import *
from src.utils import *


Mendeley 224

In [ ]:
def explorar_split_data(path):
    imagenes = {}
    for subset in ["train", "val", "test", "auto_test"]:
        subset_path = os.path.join(path, subset)
        if not os.path.exists(subset_path):
            continue
        if os.path.exists(subset_path):
            imagenes[subset] = []  # Inicializamos la lista para cada subset
            for root, dirs, files in os.walk(subset_path):
                # Omitimos el directorio raíz
                if root == subset_path:
                    continue
                imagenes[subset].append(len(files))
    print_split_table(imagenes)
    return imagenes
def print_split_table(data):
    num_clases = len(data['train'])
    tabla = []

    for i in ["train", "val", "test", "auto_test"]:
        fila = [f"{i}", data[i][0], data[i][1], data[i][2], data[i][3], data[i][4]]
        tabla.append(fila)

    print(tabulate(tabla, headers=["Clase", "0", "1", "2", "3", "4"], tablefmt="fancy_grid"))



In [ ]:
img = explorar_split_data(MENDELEY_OAI_224_SPLIT_PATH)
TOTAL = sum([sum(img[split]) for split in img])
num_por_clase = [0,0,0,0,0]
for split in img:
    print(f"Split {split}: {sum(img[split])} - {sum(img[split])/TOTAL*100:.2f}%") 
    num_por_clase = [num_por_clase[i] + img[split][i] for i in range(5)]


print(TOTAL)
print(num_por_clase)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import hashlib

def find_duplicate_images(base_path):
    # Diccionario para almacenar hashes de imágenes por clase
    hashes_by_class = {}
    # Diccionario para almacenar imágenes duplicadas por clase
    duplicates_by_class = {}
    
    # Recorre cada clase (directorio)
    for split in os.listdir(base_path):
        split_path = os.path.join(base_path, split)
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if not os.path.isdir(class_path):
                continue
            
            # Inicializa el diccionario de hashes para esta clase
            hashes = {}
            duplicates = defaultdict(list)
            
            print(f"Analizando clase: {class_name}")
            
            # Recorre cada imagen en la clase
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                if not os.path.isfile(img_path):
                    continue
                
                # Lee la imagen
                try:
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"  Error al leer: {img_path}")
                        continue
                    
                    # Calcular hash de la imagen (usando el array de bytes directamente)
                    img_hash = hashlib.md5(img.tobytes()).hexdigest()
                    
                    # Comprobar si este hash ya existe
                    if img_hash in hashes:
                        # Añadir a la lista de duplicados
                        duplicates[img_hash].append(img_name)
                        duplicates[img_hash].append(hashes[img_hash]) if hashes[img_hash] not in duplicates[img_hash] else None
                        print(f"  Imagen duplicada encontrada: {img_name} es igual a {hashes[img_hash]}")
                    else:
                        # Guardar el hash con el nombre del archivo
                        hashes[img_hash] = img_name
                except Exception as e:
                    print(f"  Error procesando {img_path}: {e}")
            
            # Guardar resultados para esta clase
            hashes_by_class[class_name] = hashes
            duplicates_by_class[class_name] = duplicates
            
            # Mostrar estadísticas para esta clase
            print(f"  Total imágenes en {class_name}: {len(hashes) + sum(len(d) for d in duplicates.values())}")
            print(f"  Imágenes únicas: {len(hashes)}")
            print(f"  Grupos de duplicados: {len(duplicates)}")
            print(f"  Total duplicados: {sum(len(d) for d in duplicates.values())}")
            print()
        
    return hashes_by_class, duplicates_by_class

# Para visualizar algunas imágenes duplicadas (opcional)
def visualize_duplicates(base_path, duplicates_by_class, max_samples=3):
    for class_name, duplicates in duplicates_by_class.items():
        if not duplicates:
            continue
            
        class_path = os.path.join(base_path, class_name)
        
        # Muestra hasta max_samples grupos de duplicados
        samples = list(duplicates.items())[:max_samples]
        
        for i, (img_hash, duplicate_names) in enumerate(samples):
            n_duplicates = len(duplicate_names)
            fig, axes = plt.subplots(1, min(n_duplicates, 5), figsize=(15, 4))
            
            if n_duplicates == 1:
                axes = [axes]  # Convertir a lista si solo hay una imagen
            
            fig.suptitle(f"Clase {class_name}: Grupo de duplicados {i+1}", fontsize=16)
            
            # Mostrar hasta 5 imágenes duplicadas
            for j, img_name in enumerate(duplicate_names[:5]):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                axes[j].imshow(img)
                axes[j].set_title(img_name)
                axes[j].axis('off')
            
            plt.tight_layout()
            plt.show()

# Ejecutar el análisis
hashes_by_class, duplicates_by_class = find_duplicate_images(MENDELEY_OAI_224_SPLIT_PATH)


In [ ]:
h, d = find_duplicate_images(MENDELEY_EXPERT1_SPLIT_PATH)

In [ ]:
d

In [ ]:
img = explorar_data(MENDELEY_EXPERT1_PATH)
total = 0

print(sum(img))

In [ ]:
img = explorar_data(MENDELEY_EXPERT2_PATH)

In [ ]:
img = explorar_split_data(MENDELEY_EXPERT1_SPLIT_PATH)
mostrar_imagenes(os.path.join(MENDELEY_EXPERT1_SPLIT_PATH, 'train'))

In [ ]:
img = explorar_split_data(MENDELEY_OAI_BRIGHT_200_SPLIT_PATH)
mostrar_imagenes(MENDELEY_OAI_BRIGHT_200_PATH)

In [ ]:
img = explorar_split_data(AUG)